In [1]:
import pandas as pd

In [10]:
with open("../collect-data/src/files/nba_data.xlsx", "rb") as file:
    nba_stats_df = pd.read_excel(file, sheet_name="NBA Stats")
    mvp_df = pd.read_excel(file, sheet_name="MVP")

In [29]:
mvp_df["MVP?"] = 1
nba_stats_df["MVP?"] = 0

mvp_dict = mvp_df.to_dict("index")
nba_stats_dict = nba_stats_df.to_dict("index")

for k in mvp_dict:
    id = mvp_dict[k]["Unnamed: 0"]
    nba_stats_dict[id]["MVP?"] = 1

nba_stats_df = pd.DataFrame.from_dict(nba_stats_dict, orient="index")
print(nba_stats_df["MVP?"].value_counts())



MVP?
0    13384
1       27
Name: count, dtype: int64
